Queries NPI Registry using their API and saves the results in an Excel File with provider information formatted and organized by use case. This program flattens the JSON data structure so that each nested value is represented as a separate column in the table.

There will be one sheet with one row of Provider Information per unique provider. This list is intended to aid a user in visualizing and tracking providers to try calling.

There will be one sheet with one row of Provider Information per unique provider+taxonomy combination. This is intended to allow a user to filter by the particular taxonomy they are looking for, e.g. "Counselor, Addiction (Substance Use Disorder)" and find all providers with that specialty. 

In [4]:
# Import required libraries
import requests
import pandas as pd

In [5]:
# Import search criteria
search_df = pd.read_csv("input.csv")
search_df

,postal_code,city,state,number,first_name,last_name
0,NaN,Sacramento,CA,NaN,NaN,NaN


In [6]:
# Create a DataFrame with only the search criteria
search_df2 = search_df.dropna(axis=1)
search_df2

,city,state
0,Sacramento,CA


In [7]:
# Convert the DataFrame to a dictionary
first_row_dict = search_df.to_dict()
print(first_row_dict)

{'postal_code': {0: nan}, 'city': {0: 'Sacramento'}, 'state': {0: 'CA'}, 'number': {0: nan}, 'first_name': {0: nan}, 'last_name': {0: nan}}


In [5]:
#Convert the used search terms to a dictionary
searchTerms = {}

col_index = 0

while col_index < len(search_df2.columns):
    # get the name of the current column
    col_name = search_df2.columns[col_index]

    # get the value of the first row in the current column
    value = search_df2[col_name][0]
    
    # assign the value of the first row to a variable with the same name as the column
    searchTerms[col_name] = value
    
    # increment the column index counter
    col_index += 1

# print the resulting dictionary of variables
print(searchTerms)

{'city': 'Sacramento', 'state': 'CA'}


In [8]:
# Use the search terms to append the api URL
searchTerms_input = {}

col_index = 0

npi_url_base = "https://npiregistry.cms.hhs.gov/api/?version=2.1&use_first_name_alias=True&pretty=TRUE&enumeration_type=NPI-1&limit=100"
npi_url = npi_url_base

while col_index < len(search_df2.columns):
    # get the name of the current column
    col_name = search_df2.columns[col_index]

    # get the value of the first row in the current column
    value = search_df2[col_name][0]
    
    # create the search term suffix
    term = "&"+col_name+"="+value
    print(term)
    
    npi_url = npi_url + term

    # increment the column index counter
    col_index += 1
    
print(npi_url)

&city=Sacramento
&state=CA
https://npiregistry.cms.hhs.gov/api/?version=2.1&use_first_name_alias=True&pretty=TRUE&enumeration_type=NPI-1&limit=100&city=Sacramento&state=CA


In [9]:
# Open the csv with all the appropriate taxonomies. Used to filter only for mental health providers

taxonomies = pd.read_csv("NPI_Taxonomies.csv", usecols=['Taxonomies'])
taxonomies = taxonomies[taxonomies.columns[0]].values.tolist()
taxonomies = [x for x in taxonomies if not(pd.isnull(x)) == True]
print(taxonomies)

['Marriage & Family Therapist', 'Psychoanalyst', 'Psychologist', 'Social Worker', 'Marriage and ']


In [11]:
# Create a dictionary of URLs with one version per Taxonomy, to perform multiple API calls (since you cannot search for multiple taxonomies at once)

v = 1 # URL version number
url_versions = {} # empty dictionary to store different versions of the URLs based on taxonomy
for t in taxonomies:
    # create the search term suffix
    term = "&taxonomy_description="+t
    
    # create the URL
    url = npi_url+term

    # save the URL as a new dictionary item
    url_versions[v] = url
    
    # increase the counter
    v+= 1
    
    print(url)
    
print(url_versions)

https://npiregistry.cms.hhs.gov/api/?version=2.1&use_first_name_alias=True&pretty=TRUE&enumeration_type=NPI-1&limit=100&city=Sacramento&state=CA&taxonomy_description=Marriage & Family Therapist
https://npiregistry.cms.hhs.gov/api/?version=2.1&use_first_name_alias=True&pretty=TRUE&enumeration_type=NPI-1&limit=100&city=Sacramento&state=CA&taxonomy_description=Psychoanalyst
https://npiregistry.cms.hhs.gov/api/?version=2.1&use_first_name_alias=True&pretty=TRUE&enumeration_type=NPI-1&limit=100&city=Sacramento&state=CA&taxonomy_description=Psychologist
https://npiregistry.cms.hhs.gov/api/?version=2.1&use_first_name_alias=True&pretty=TRUE&enumeration_type=NPI-1&limit=100&city=Sacramento&state=CA&taxonomy_description=Social Worker
https://npiregistry.cms.hhs.gov/api/?version=2.1&use_first_name_alias=True&pretty=TRUE&enumeration_type=NPI-1&limit=100&city=Sacramento&state=CA&taxonomy_description=Marriage and 
{1: 'https://npiregistry.cms.hhs.gov/api/?version=2.1&use_first_name_alias=True&pretty=

In [12]:
# Search NPI registry and store results

i = 1
addresses_df = pd.DataFrame()
basic_info_df = pd.DataFrame()
taxonomies_df = pd.DataFrame()
result_df = pd.DataFrame()

for url_version in url_versions.values():
    # get the requests from the URL
    response = requests.get(url_version)    

    # If there are no results for that taxonomy, it will make a successful request but with an error in the result. 
    # Simply skip this search term
    if "Errors" in response.json():
        print("Version",i,": No Results")
    
    # Otherwise, response status code 200 means it has succeeded; will only continue if the API call is successful
    elif response.status_code == 200:
        print("Version",i,": Success")
        # convert addresses of result to DF
        addresses_df1 = pd.json_normalize(
            response.json()["results"], 
            record_path=['addresses'], # Looks in the address path of the json
            meta=['number'], # Uses the NPI number column as metadata for each record
            errors='ignore') # Avoid raising a key error if key isn't present
        addresses_df = pd.concat([addresses_df, addresses_df1], ignore_index=True)
        #basic_info_df = pd.json_normalize(response.json()["basic"], errors='ignore')
        basic_info_df1 = pd.json_normalize(
            response.json(),['results'], # Looks in the basic_info path of the json
            errors='ignore')
        basic_info_df = pd.concat([basic_info_df, basic_info_df1], ignore_index=True)
        taxonomies_df1 = pd.json_normalize(
            response.json()["results"], 
            record_path=['taxonomies'],
            meta=['number'], # Uses the NPI number column as metadata for each record
            errors='ignore')
        taxonomies_df = pd.concat([taxonomies_df, taxonomies_df1], ignore_index=True)
    # Otherwise, print the error code for troublelshooting
    else:
        print("Version",i,"Error: API request returned status code ",response.status_code)
    # increase the counter
    i+= 1

Version 1 : Success
Version 2 : Success
Version 3 : Success
Version 4 : Success
Version 5 : No Results


In [16]:
# Merge all of the DataFrames created in the API search above
result_df = addresses_df.merge(basic_info_df, on='number')
result_df = result_df.merge(taxonomies_df, on='number')
result_df

,country_code,country_name,address_purpose,address_type,address_1,city,state_x,postal_code,telephone_number,fax_number,...,basic.name_prefix,basic.name_suffix,basic.credential,basic.certification_date,code,taxonomy_group,desc,state_y,license,primary
0,US,United States,MAILING,DOM,8912 VOLUNTEER LN,SACRAMENTO,CA,958263221,916-344-0199,916-344-0196,...,Miss,--,NaN,NaN,106H00000X,,Marriage & Family Therapist,CA,MFTI 62210,True
1,US,United States,LOCATION,DOM,8912 VOLUNTEER LN,SACRAMENTO,CA,958263221,916-344-0199,916-344-0196,...,Miss,--,NaN,NaN,106H00000X,,Marriage & Family Therapist,CA,MFTI 62210,True
2,US,United States,LOCATION,DOM,2020 29TH ST,SACRAMENTO,CA,958171119,916-495-4561,NaN,...,NaN,NaN,LMFT,2022-12-23,101Y00000X,,Counselor,None,None,False
3,US,United States,LOCATION,DOM,2020 29TH ST,SACRAMENTO,CA,958171119,916-495-4561,NaN,...,NaN,NaN,LMFT,2022-12-23,106H00000X,,Marriage & Family Therapist,CA,MFT119187,True
4,US,United States,MAILING,DOM,2020 29TH ST STE 205,SACRAMENTO,CA,958171119,916-495-4561,916-706-0929,...,NaN,NaN,LMFT,2022-12-23,101Y00000X,,Counselor,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,US,United States,LOCATION,DOM,2118 P ST,SACRAMENTO,CA,958166149,916-307-8476,NaN,...,Dr.,--,LCSW,NaN,1041C0700X,,"Social Worker, Clinical",CA,LCS 18019,False
998,US,United States,MAILING,DOM,7300 WYNDHAM DR,SACRAMENTO,CA,958234913,916-525-6100,NaN,...,--,--,NaN,NaN,1041C0700X,,"Social Worker, Clinical",CA,21402,True
999,US,United States,LOCATION,DOM,7300 WYNDHAM DR,SACRAMENTO,CA,958234913,916-525-6100,NaN,...,--,--,NaN,NaN,1041C0700X,,"Social Worker, Clinical",CA,21402,True
1000,US,United States,MAILING,DOM,1116 22ND ST,SACRAMENTO,CA,958164912,916-443-2546,916-443-6304,...,Ms.,--,MSW,NaN,1041C0700X,,"Social Worker, Clinical",CA,LCS6324,True


In [17]:
#Rename columns for easier readability

result_df.rename(columns={"number":"NPI_Number",
                          "basic.first_name": "First_Name",
                          "basic.last_name": "Last_Name",
                          "basic.middle_name": "Middle_Name",
                          "basic.gender": "Gender",
                          "basic.credential": "Credential",
                          "basic.sole_proprietor": "Sole_Proprietor",
                          "telephone_number": "Phone_Number",
                          "desc": "Taxonomy",
                          "state_x":"State",
                          "enumeration_type": "Enumeration_Type",
                          "city":"City",
                          "postal_code":"Zip"
                         }, inplace=True)

In [18]:
result_df.head()

,country_code,country_name,address_purpose,address_type,address_1,City,State,Zip,Phone_Number,fax_number,...,basic.name_prefix,basic.name_suffix,Credential,basic.certification_date,code,taxonomy_group,Taxonomy,state_y,license,primary
0,US,United States,MAILING,DOM,8912 VOLUNTEER LN,SACRAMENTO,CA,958263221,916-344-0199,916-344-0196,...,Miss,--,NaN,NaN,106H00000X,,Marriage & Family Therapist,CA,MFTI 62210,True
1,US,United States,LOCATION,DOM,8912 VOLUNTEER LN,SACRAMENTO,CA,958263221,916-344-0199,916-344-0196,...,Miss,--,NaN,NaN,106H00000X,,Marriage & Family Therapist,CA,MFTI 62210,True
2,US,United States,LOCATION,DOM,2020 29TH ST,SACRAMENTO,CA,958171119,916-495-4561,NaN,...,NaN,NaN,LMFT,2022-12-23,101Y00000X,,Counselor,None,None,False
3,US,United States,LOCATION,DOM,2020 29TH ST,SACRAMENTO,CA,958171119,916-495-4561,NaN,...,NaN,NaN,LMFT,2022-12-23,106H00000X,,Marriage & Family Therapist,CA,MFT119187,True
4,US,United States,MAILING,DOM,2020 29TH ST STE 205,SACRAMENTO,CA,958171119,916-495-4561,916-706-0929,...,NaN,NaN,LMFT,2022-12-23,101Y00000X,,Counselor,None,None,False


In [19]:
# Select columns to keep (instead of drop, in case NPI adds columns later)

result_df = result_df[['NPI_Number','Enumeration_Type','address_purpose','City', 'State', 'Zip',
       'Phone_Number','First_Name', 'Last_Name', 'Middle_Name', 'Credential',
       'Sole_Proprietor', 'Gender','basic.status','Taxonomy']]
result_df.head()

,NPI_Number,Enumeration_Type,address_purpose,City,State,Zip,Phone_Number,First_Name,Last_Name,Middle_Name,Credential,Sole_Proprietor,Gender,basic.status,Taxonomy
0,1134441124,NPI-1,MAILING,SACRAMENTO,CA,958263221,916-344-0199,LINDA,ABDELBAKI,KAMAL,NaN,NO,F,A,Marriage & Family Therapist
1,1134441124,NPI-1,LOCATION,SACRAMENTO,CA,958263221,916-344-0199,LINDA,ABDELBAKI,KAMAL,NaN,NO,F,A,Marriage & Family Therapist
2,1972056471,NPI-1,LOCATION,SACRAMENTO,CA,958171119,916-495-4561,RABIIA,ABDUL-ALI,NaN,LMFT,YES,F,A,Counselor
3,1972056471,NPI-1,LOCATION,SACRAMENTO,CA,958171119,916-495-4561,RABIIA,ABDUL-ALI,NaN,LMFT,YES,F,A,Marriage & Family Therapist
4,1972056471,NPI-1,MAILING,SACRAMENTO,CA,958171119,916-495-4561,RABIIA,ABDUL-ALI,NaN,LMFT,YES,F,A,Counselor


In [20]:
# Clean the DataFrame
    #Keep only Mailing address lines, to remove duplicate entries and keep only Active NPIs. Remove columns no longer needed like status.
result_df_mail = result_df.loc[result_df['address_purpose'] == 'MAILING']
result_df_status = result_df_mail.loc[result_df_mail['basic.status'] == 'A']
result_df_2 = result_df_status.drop(columns=['Enumeration_Type', 'address_purpose','basic.status'])
result_df_2.head()

,NPI_Number,City,State,Zip,Phone_Number,First_Name,Last_Name,Middle_Name,Credential,Sole_Proprietor,Gender,Taxonomy
0,1134441124,SACRAMENTO,CA,958263221,916-344-0199,LINDA,ABDELBAKI,KAMAL,NaN,NO,F,Marriage & Family Therapist
4,1972056471,SACRAMENTO,CA,958171119,916-495-4561,RABIIA,ABDUL-ALI,NaN,LMFT,YES,F,Counselor
5,1972056471,SACRAMENTO,CA,958171119,916-495-4561,RABIIA,ABDUL-ALI,NaN,LMFT,YES,F,Marriage & Family Therapist
6,1346340577,SACRAMENTO,CA,958171337,916-876-5205,TAKESHI,ABE,NaN,MFT,NO,M,Marriage & Family Therapist
9,1427365360,SACRAMENTO,CA,958192704,NaN,KATHERINE,ABELL,O'BRIEN,NaN,NO,F,Marriage & Family Therapist


In [23]:
# Create dummy variables from the "Taxonomy" column

result_df_taxonomy_search = pd.get_dummies(result_df_2, columns=['Taxonomy'], prefix="Taxonomy")
result_df_taxonomy_search.head()

,NPI_Number,City,State,Zip,Phone_Number,First_Name,Last_Name,Middle_Name,Credential,Sole_Proprietor,...,"Taxonomy_Psychologist, Forensic","Taxonomy_Psychologist, Group Psychotherapy","Taxonomy_Psychologist, Intellectual & Developmental Disabilities","Taxonomy_Psychologist, Rehabilitation","Taxonomy_Psychologist, School",Taxonomy_Rehabilitation Counselor,Taxonomy_Social Worker,"Taxonomy_Social Worker, Clinical","Taxonomy_Social Worker, School",Taxonomy_Student in an Organized Health Care Education/Training Program
0,1134441124,SACRAMENTO,CA,958263221,916-344-0199,LINDA,ABDELBAKI,KAMAL,NaN,NO,...,0,0,0,0,0,0,0,0,0,0
4,1972056471,SACRAMENTO,CA,958171119,916-495-4561,RABIIA,ABDUL-ALI,NaN,LMFT,YES,...,0,0,0,0,0,0,0,0,0,0
5,1972056471,SACRAMENTO,CA,958171119,916-495-4561,RABIIA,ABDUL-ALI,NaN,LMFT,YES,...,0,0,0,0,0,0,0,0,0,0
6,1346340577,SACRAMENTO,CA,958171337,916-876-5205,TAKESHI,ABE,NaN,MFT,NO,...,0,0,0,0,0,0,0,0,0,0
9,1427365360,SACRAMENTO,CA,958192704,NaN,KATHERINE,ABELL,O'BRIEN,NaN,NO,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# Remove Duplicates (and the taxonomy column) to get a list of unique providers
result_df_unique = result_df_2.drop_duplicates(subset="NPI_Number", keep='first', inplace=False, ignore_index=False)
result_df_unique = result_df_unique.dropna(subset=['Phone_Number'])
result_df_unique

,NPI_Number,City,State,Zip,Phone_Number,First_Name,Last_Name,Middle_Name,Credential,Sole_Proprietor,Gender,Taxonomy
0,1134441124,SACRAMENTO,CA,958263221,916-344-0199,LINDA,ABDELBAKI,KAMAL,NaN,NO,F,Marriage & Family Therapist
4,1972056471,SACRAMENTO,CA,958171119,916-495-4561,RABIIA,ABDUL-ALI,NaN,LMFT,YES,F,Counselor
6,1346340577,SACRAMENTO,CA,958171337,916-876-5205,TAKESHI,ABE,NaN,MFT,NO,M,Marriage & Family Therapist
12,1619084043,ELK GROVE,CA,957578185,916-685-3238,CHARLOTTE,ADAMS,NaN,MFT INTERN,NO,F,Marriage & Family Therapist
14,1538323712,GOLD RIVER,CA,956704444,916-956-5736,CORY,ADAMS,SCOTT,NaN,YES,M,Marriage & Family Therapist
...,...,...,...,...,...,...,...,...,...,...,...,...
990,1770874075,SACRAMENTO,CA,958171353,916-734-5076,JORGE,BECERRA,NaN,NaN,NO,M,Social Worker
992,1538452289,SACRAMENTO,CA,958202163,916-734-2458,SUSANA,BECERRA,NaN,LCSW,YES,F,"Social Worker, Clinical"
994,1295867216,SACRAMENTO,CA,958184306,916-443-5287,ANDREW,BEIN,MARTIN,LCSW,YES,M,"Social Worker, Clinical"
998,1174687032,SACRAMENTO,CA,958234913,916-525-6100,VIELKA,BELLOSO,EDITH,NaN,YES,F,"Social Worker, Clinical"


In [25]:
# Save the results to an Excel file with one sheet per DataFrame above.
with pd.ExcelWriter('NPI_Provider_Info.xlsx') as writer:  
    result_df_unique.to_excel(writer, sheet_name='Unique_Provider_List')
    result_df_taxonomy_search.to_excel(writer, sheet_name='Providers_by_Taxonomy')